In [1]:
"""
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
"""

# Chat
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOllama

# Prompt
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Memory
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(
temperature=0.1,
)

llm = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
)

In [2]:
examples = [
    {
        "question": "How can you change the title of 'Topgun' to emoji?",
		"answer": "🛩️👨‍✈️🔥"
    },

    {
        "question": "How can you change the title of 'Godfather' to emoji?",
		"answer": "👨‍👨‍👦🔫🍝"
    },

    {
        "question": "How can you change the title of 'HarryPoter' to emoji?",
		"answer": "🔮🏰🧙"
    },
]


example_prompt = PromptTemplate.from_template(
    """
    You are a movieholic and know every scenario of the movie. 
    You have to change the titile of movie to emogi, considering the movie's concept. 
    You must only use 3 emojis to represent the movie without any texts. Not explanation, Not note, You must use only 3 emojis in this responce.
    
    
    Human: {question}
    AI: {answer}
    """
)

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
	suffix="How can you change the title of {movie} to emoji?",
	input_variables=["movie"]
)

chain = prompt | llm

In [3]:
memory = ConversationBufferMemory(
	llm = llm,
	max_token_limit = 100,
	memory_key='chat_history',
	return_messages = True,
	)

def load_memory():
	return memory.load_memory_variables({})["chat_history"]

In [4]:
def invoke_chain(title):
    m_title = title
    result = chain.invoke({"movie": m_title})
    print(result.content)
    
    memory.save_context({"input":f"{m_title}"}, {"output":result.content},)

def get_result(title):

    if load_memory() == [] :
        result = invoke_chain(title)
        
        print(result)
        print('\n')
        print('[Recent Search]')
        print('No history')

    else:
        result = invoke_chain(title)

        print(result)
        print('\n')
        print('[Recent Search]')
        print(str(load_memory()[-4]).replace("content='","").replace("'",""), ":", str(load_memory()[-3]).replace("content='","").replace("'",""))


In [5]:
get_result('Aquaman')

 🌊� kennis 🦈 (or 🐬 for dolphin if you prefer)
None


[Recent Search]
No history


In [6]:
get_result('Superman')

 👨‍🚀🔵💥

(Note: The blue S represents the "S" in Superman.)
None


[Recent Search]
Aquaman :  🌊� kennis 🦈 (or 🐬 for dolphin if you prefer)


In [7]:
get_result('Spiderman')

 🕷️👨‍⬛💥
None


[Recent Search]
Superman :  👨\u200d🚀🔵💥\n\n(Note: The blue S represents the "S" in Superman.)
